In [6]:
import pandas as pd
import easyocr
import math

In [36]:
def extract_table_easyocr(image_path, price=None):
    """Extract table using EasyOCR - clean version without debug output"""
    try:
        # Initialize EasyOCR reader
        reader = easyocr.Reader(['en'])
        
        # Read text from image
        results = reader.readtext(image_path)
        
        # Group text by vertical position (rows)
        text_boxes = []
        for (bbox, text, confidence) in results:
            if confidence > 0.5:  # Filter out low confidence results
                # Get bounding box coordinates
                top_left = bbox[0]
                bottom_right = bbox[2]
                
                text_boxes.append({
                    'text': text,
                    'x': top_left[0],
                    'y': top_left[1],
                    'width': bottom_right[0] - top_left[0],
                    'height': bottom_right[1] - top_left[1],
                    'confidence': confidence
                })
        
        if not text_boxes:
            return pd.DataFrame()
        
        # Sort by Y coordinate (top to bottom)
        text_boxes.sort(key=lambda x: x['y'])
        
        # Group text boxes into rows based on Y coordinate
        rows = []
        current_row = []
        current_y = text_boxes[0]['y']
        tolerance = 15
        
        for box in text_boxes:
            if abs(box['y'] - current_y) <= tolerance:
                current_row.append(box)
            else:
                if current_row:
                    # Sort current row by X coordinate (left to right)
                    current_row.sort(key=lambda x: x['x'])
                    row_text = [box['text'] for box in current_row]
                    rows.append(row_text)
                current_row = [box]
                current_y = box['y']
        
        # Add the last row
        if current_row:
            current_row.sort(key=lambda x: x['x'])
            row_text = [box['text'] for box in current_row]
            rows.append(row_text)
        
        if rows:
            # Make all rows the same length
            max_cols = max(len(row) for row in rows)
            
            # Pad all rows to have the same number of columns
            for row in rows:
                row.extend([''] * (max_cols - len(row)))
            headers = ["Strike", "Divider", "Bid", "Ask", "Last", "Delta", "IV", "Volume"]
            df = pd.DataFrame(rows, columns=headers)
            # remove divider column
            df = df.drop(columns=['Divider'])
            
            # Convert all columns to float
            for col in df.columns:
                # Clean the data first (remove % signs, etc.)
                df[col] = df[col].astype(str).str.replace('%', '').str.replace(',', '')
                # Convert to float, replacing any unconvertible values with NaN
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Set Strike as index
            df = df.set_index('Strike')
            
            # Calculate Buffer column if price is provided
            if price is not None:
                E = (df['IV'] / math.sqrt(52)) * 0.017
                D = price * (1 - E)
                df['Buffer (95.5% Confidence)'] = round(D - df.index, 1)

            return df
        
        return pd.DataFrame()
    
    except Exception as e:
        return pd.DataFrame()

# Extract table from your image
price = 222.65
df = extract_table_easyocr("/Users/dylan-mini/Documents/code/Nero/src/data/test.png", price=price)

/Users/dylan-mini/Documents/code/Nero/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [42]:
df

,Bid,Ask,Last,Delta,IV,Volume,Buffer (95.5% Confidence)
Strike,,,,,,,
75,0.05,0.10,0.06,-0.0026,267.89,236,7.0
80,0.00,0.15,0.15,-0.0034,259.87,31,6.2
85,0.10,0.30,0.14,-0.0045,252.34,45,5.2
90,0.00,0.40,0.20,-0.0058,245.24,96,3.9
95,0.15,0.25,0.22,-0.0074,238.52,99,2.5
100,0.25,0.30,0.27,-0.0100,235.00,723,-0.7
105,0.10,1.25,0.40,-0.0128,229.90,35,-3.0
110,0.30,0.60,0.45,-0.0162,225.03,34,-5.5
115,0.15,1.00,0.69,-0.0187,215.84,23,-5.6


In [64]:
def calculate_option_metrics(df, strike_price, premium):
    """
    Calculate Max Loss and Return on Risk Percentage for a specific option trade
    
    Args:
        df: DataFrame with options data
        strike_price: The strike price of the option
        premium: The premium paid/received for the option
    
    Returns:
        max_loss and return_on_risk_percentage
    """
    # Calculate Max Loss = (Strike - premium) * 100
    max_loss = (strike_price - premium) * 100
    
    # Calculate Return on Risk Percentage = (premium / max_loss) * 100
    return_on_risk_percentage = round(((premium * 100) / max_loss) * 100, 2) if max_loss != 0 else 0
    
    
    return f"${round(max_loss)}", f"{return_on_risk_percentage}%"


In [67]:
calculate_option_metrics(df, 95, 0.25)

('$9475', '0.26%')